In [3]:
from math import sqrt
import numpy as np
import numpy.linalg as nla

In [369]:
def householder(x,i):
    
    n = x.shape[0]
    dot_1on = np.dot(x,x) - x[i]**2

    v = np.copy(x)
    v[i] = 1.0
    
    if dot_1on < np.finfo(float).eps:
        beta = 0.0
    else:
        norm_x = sqrt(x[i]**2 + dot_1on)
        if x[i] <= 0:
            v[i] = x[i] - norm_x
        else:
            v[i] = -dot_1on / (x[i] + norm_x)
        beta = 2 * v[i]**2 / (dot_1on + v[i]**2)
        v = v / v[i]
    
    return np.eye(n) - beta * np.outer(v,v)


In [406]:
# polyharmonic spline rbf
def phs(d,k):
    
    if k%2 == 0:
        if d == 0:
            return 0
        else:
            return (d**k) * np.log(d)
    else:
        return d**k

# 1st order differential operator of polyharmonic spline rbf
def diff_phs(d,d_i,k):
    
    if k%2 == 0:
        if d == 0:
            df = 0
        else:
            df = d_i * (d**(k-2)) * (1 + (k * np.log(d)))
    else:
        df = k * d_i * (d**(k-2))

    return df

def GA_rbf(d,ep):
    
    return np.exp(-(ep*d)**2)

def L_GA_rbf(d,ep,k):
    
    epr2 = (ep*d)**2
    
    p = np.ones((k+1,d.shape[0]))
    p[1] = 4*epr2 - 4
    
    for i in range(1,k):
        p[i+1] = 4*(epr2 - (2*i + 1))*p[i] - (16*i**2)*p[i-1]
    
    return ep**(2*k) * p[k] * GA_rbf(d,ep)

In [1]:
def get_rbffd_weights(X,D,N):
    
    k = 3
    n = X.shape[1]
    
    # apply householder transformation to center stencil on x-axis
    H = householder(X[:,0],2)
    Xp = np.matmul(H,X)
    
    # Create A Matrix
    A_rbf = phs(D,k)
    
    # Polynomial part
    N_poly = 21
    
    x = Xp[0].reshape((1,-1))
    y = Xp[1].reshape((1,-1))
    A_poly = np.concatenate((np.ones((1,n)),x,y,x**2,x*y,y**2,x**3,(x**2)*y,x*(y**2),y**3,x**4,(x**3)*y,(x**2)*(y**2),x*(y**3),y**4,x**5,(x**4)*y,(x**3)*(y**2),(x**2)*(y**3),x*(y**4),y**5),axis=0)
    
    
    # create full PHS-RBF Matrix
    A = np.concatenate((np.concatenate((A_rbf,np.transpose(A_poly)),axis=1),np.concatenate((A_poly,np.zeros((N_poly,N_poly))),axis=1)),axis=0)

    # for d/dx
    b_x = np.concatenate((diff_phs(D[0],x[0,0]-x[0],k),np.eye(1,N_poly,1).reshape((-1))),axis=0)
    w_x = nla.solve(A,b_x)[0:n]
    
    # for d/dy
    b_y = np.concatenate((diff_phs(D[0],y[0,0]-y[0],k),np.eye(1,N_poly,2).reshape((-1))),axis=0)
    w_y = nla.solve(A,b_y)[0:n]
    
    # reverse householder transformation to get final weights
    d1h3p = np.concatenate((w_x,w_y,np.zeros(n)),axis=0).reshape((3,-1))
    d1h3 = np.matmul(H,d1h3p)
         
    d1v3p = np.append(np.zeros((2,7)),np.array([[-1/60,3/20,-3/4,0,3/4,-3/20,1/60]]),axis=0)
    d1v3 = np.matmul(H,d1v3p)
    
    ### Hyperviscosity
    ep = 0.044*np.sqrt(N) - 0.14
    kk = 4
    
    ### horizontal component
    # Create A Matrix
    A_rbf = GA_rbf(D,ep)
    
    b_L = L_GA_rbf(D[0],ep,kk)
    w_L = nla.solve(A_rbf,b_L)
    
    return d1h3,d1v3,w_L

In [5]:
def get_RBFFD_DMs(X,n):
    # xx: 2D FP array (3xNnodes) describing the nodeset in cartesian coordinates
    # n: stencil size
    
    # get number of nodes
    Nh = X.shape[1]
    
    # tensor product to "meshgrid" coordinates
    xx,yy,zz = np.tensordot(X, np.ones(Nh), 0)
    
    # distance matrix
    D = np.sqrt((xx - xx.T)**2 + (yy - yy.T)**2 + (zz - zz.T)**2)
    
    # get n-point stencils
    idx = np.argsort(D)[:,0:n]
    
    # initialize DMs
    DM1h3 = np.empty((3,Nh,n))
    DM1v3 = np.empty((3,Nh,7))
    Lh = np.empty((Nh,n))
    
    # for each stencil get associated RBFFD differentiation weights
    for i in range(Nh):
        
        d1h3,d1v3,Lh[i] = get_rbffd_weights(X[:,idx[i]],(D[idx[i]])[:,idx[i]],Nh)
        
        for j in range(3):
            DM1h3[j,i,:] = d1h3[j,:]
            DM1v3[j,i,:] = d1v3[j,:]
        
        if i%np.minimum(1000,int(Nh/5)) == 0:
            print("get_RBFFD_DMs progress: %{0:d}".format(int((100*i)/Nh)+1))
            
    return DM1h3,DM1v3,Lh,idx


4.7 6.764357013041135e-15
